In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import confusion_matrix

In [3]:
data = fetch_20newsgroups()
categories = data.target_names
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

In [8]:
# Create a set of all unique words in the training data
vocab_set = set()
for dstr in train.data:
    words = dstr.lower().split()
    vocab_set.update(words)

# Convert the set of unique words to a list
new_vocab= list(vocab_set)

In [9]:
# FOR TRAIN
new_data_category = {category: [] for category in categories}
new_vocabcount_category = {category: {v: 0 for v in new_vocab} for category in categories}

for i in range(len(train.data)):
    category = train.target_names[train.target[i]]
    if category in categories:
        new_data_category[category].append(train.data[i])

for category, data in new_data_category.items():
    total_words = 0
    for dstr in data:
        words = dstr.lower().split()
        for w in words:
            if w in new_vocab:
                total_words += 1
                new_vocabcount_category[category][w] += 1

    for k in new_vocabcount_category[category]:
        new_vocabcount_category[category][k] += 1

    total = sum(new_vocabcount_category[category].values())
    for k in new_vocabcount_category[category]:
        new_vocabcount_category[category][k] /= total

new_vocab_logp_category = {category: {key: np.log(value) for key, value in vocabcount.items()} 
                           for category, vocabcount in new_vocabcount_category.items()}

In [ ]:
# FOR TEST
new_predicted_count = 0
new_predicted_labels = []

for i in range(len(test.data)):
    tsample = test.data[i]
    new_logp_category = {category: sum(new_vocab_logp_category[category].get(w, 0) 
                                       for w in tsample.lower().split() if w in new_vocab) 
                         for category in categories}

    new_predicted_label = max(new_logp_category, key=new_logp_category.get)
    new_predicted_labels.append(new_predicted_label)

    true_label = test.target_names[test.target[i]]
    if new_predicted_label == true_label:
        new_predicted_count += 1

new_test_accuracy = new_predicted_count / len(test.data)
print("New Test Accuracy(%):", new_test_accuracy * 100)
